In [ ]:
import datetime as dt
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path
from typing import List

import utm
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry as sg
import shapely.ops as so

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [ ]:
path = DATA_DIR/'cairns_gtfs.zip'
feed = gk.read_feed(path, dist_units='km')
feed.describe()

In [ ]:
feed.validate()

In [ ]:
g = feed.geometrize_routes(use_utm=True)

In [ ]:
feed.map_trips(trip_ids[:1], include_stops=True, include_arrows=True)

In [ ]:
dates = feed.get_first_week()[6:]
activity = feed.compute_trip_activity(dates)
display(activity)

trip_ids = activity.loc[
    lambda x: x.filter(dates).sum(axis=1) > 0,
    "trip_id",
]
trip_ids

In [ ]:
feed.trips

In [ ]:
dates = feed.get_first_week()[:1] + ["20010101"]

path = DATA_DIR / "cairns_screen_lines.geojson"
screen_lines = gpd.read_file(path)
f = feed.compute_screen_line_counts(screen_lines, dates)
f

In [ ]:
path = DATA_DIR/'cairns_gtfs.zip'
gk.list_feed(path)



In [ ]:
feed = (
    gk.read_feed(path, dist_units='km')
    .append_dist_to_stop_times()
)
feed.describe()

In [ ]:
shapes = feed.geometrize_shapes(use_utm=True)
shapes.crs
screen_lines = screen_lines.to_crs(shapes.crs)
screen_lines

In [ ]:
url = "http://transitfeeds.com/p/10-15-transit/936/latest/download"
gk.read_feed(url, dist_units="km").describe()

In [ ]:
(
    g1.crossing_time.equals(g2.crossing_time),
    g1.crossing_direction.equals(g2.orientation)
)

In [ ]:
feed.map_routes(feed.routes.route_id.iloc[:4], include_stops=False)
